**Set-up instructions:** this notebook gives a tutorial of transformation algorithms supported by `sktime`.
On binder, this should run out-of-the-box.

To run this notebook as intended, ensure that `sktime` with basic dependency requirements is installed in your python environment.

To run this notebook with a local development version of sktime, either uncomment and run the below, or `pip install -e` a local clone of the `sktime` `main` branch.

In [ ]:
# from os import sys
# sys.path.append("..")

# Using sktime transformers

Transformers are modularized data processing steps commonly used in machine learning. `sktime` introduces transformer types related to time series, which can be used in pipeline constructs across learning tasks. For instance, the same time series feature extractor could be used in a forecasting pipeline, or a time series classification pipeline.

Note: the term "transformer" is used in the sense of `scikit-learn`. This should be distinguished from the "transformer" referring to specific neural network architecture.

`sktime` provides common interfaces to different types of transformations for stand-alone or composition use. As all `sktime` estimators, transformers expose parameters which can be tuned inside pipelines and composites.

**Section 1** provides am introductory overview of transformers in `sktime` and their usage.

**Section 2** introduces the different kinds of transformers in technical detail.

**Section 3** discusses use of transformers in composites such as pipelines or algorithms that require distances or kernels.

**Section 4** gives an introduction to how to write custom transformers compliant with the `sktime` interface.

## Table of Contents

* [1. Introduction to transformers in sktime](#chapter1)
    * [1.1 What are transformers?](#section_1_1)
    * [1.2 Different types of transformers](#section_1_2)   
    * [1.3 Broadcasting aka vectorization of transformers](#section_1_3)         
    * [1.4 Transformers as pipeline components](#section_1_4)
* [2. Transformer types and signatures](#chapter2)
    * [2.1 Data container format](#section_2_1)
    * [2.2 General transformer signature - time series transformers](#section_2_2)   
    * [2.3 General transformer signature - pairwise series transformers](#section_2_3)         
    * [2.4 General transformer signature - pairwise transformers](#section_2_4)
    * [2.5 Searching for transformers](#section_1_5)
* [3. Transformers in pipelines and composition](#chapter3)  
* [4. Extension guide - implementing your own transformer](#chapter4)        
* [5. Summary](#chapter5)          

#### package imports

## 1. Introduction to transformers in `sktime` <a class="anchor" id="chapter1"></a>

This section introduces transformers in `sktime` and their main features and use cases:

* transformers are a unified interface for data processing steps with `fit` and `transform`
* there are different types of transformers in `sktime`, e.g., with different output type - time series or scalar features
* transformers broadcast/vectorize across time series instances and variables
* all transformers can be used as components in pipelines of any (fitting) type, e.g., forecasting or classifier pipelines

### 1.1 What are transformers? <a class="anchor" id="section_1_1"></a>

Transformers are modularized data processing steps commonly used in machine learning, the term "transformer" used in the sense of `scikit-learn`.

Transformers are estimators that:

* are fitted to a batch of data via `fit`, changing its state
* are applied to another batch of data via `transform`, producing transformed data

In `sktime`, input to `fit` and `transform` is typically a time series.

Basic use of an `sktime` time series transformer is as follows:

In [ ]:
# 1. prepare the data
from sktime.utils._testing.series import _make_series

X = _make_series()
X_train = X[:30]
X_test = X[30:]
# X_train and X_test are both pandas.Series

X_train, X_test

In [ ]:
# 2. construct the transformer
from sktime.transformations.series.boxcox import BoxCoxTransformer

# trafo is an sktime estimator inheriting from BaseTransformer
# Box-Cox transform with lambda parameter fitted via mle
trafo = BoxCoxTransformer(method="mle")

In [ ]:
# 3. fit the transformer to training data
trafo.fit(X_train)

# 4. apply the transformer to transform test data
# Box-Cox transform with lambda fitted on X_train
X_transformed = trafo.transform(X_test)

X_transformed

If the training and test set is the same, step 3 and 4 can be carried out more concisely (and sometimes more efficiently) by using `fit_transform`:

In [ ]:
# 3+4. apply the transformer to fit and transform on the same data, X
X_transformed = trafo.fit_transform(X)

### 1.2 Different types of transformers <a class="anchor" id="section_1_2"></a>

`sktime` distinguishes different types of transformer, depending on the input type of `fit` and `transform`, and the output type of `transform`.

Transformers differ by:

* making use of an additional `y` argument in `fit` or `transform`
* whether the input to `fit` and `transform` is a single time series, a collection of time series, or scalar values (data frame row)
* whether the output of `transform` is a single time series, a collection of time series, or scalar values (data frame row)
* whether the input to `fit` and `transform` are one object or two. Two objects as input and a scalar output means the transformer is a distance or kernel function.

More detail on this is given in [Section 2](#chapter2).

To illustrate the difference, we compare two transformers with different output:

* the Box-Cox transformer `BoxCoxTrannsformer`, which transforms a time series to a time series
* the summary transformer `SummaryTransformer`, which transforms a time series to scalars such as the mean


In [ ]:
# constructing the transformer
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.transformations.series.summarize import SummaryTransformer
from sktime.utils._testing.series import _make_series

# getting some data
# this is one pandas.Series
X = _make_series(n_timepoints=10)

# constructing the transformers
boxcox_trafo = BoxCoxTransformer(method="mle")
summary_trafo = SummaryTransformer()

In [ ]:
# this produces a pandas Series
boxcox_trafo.fit_transform(X)

In [ ]:
# this produces a pandas.DataFrame row
summary_trafo.fit_transform(X)

For time series transformers, the metadata tags describe the expected output of `transform`:

In [ ]:
boxcox_trafo.get_tag("scitype:transform-output")

In [ ]:
summary_trafo.get_tag("scitype:transform-output")

A more complete overview on transformer types and tags is given in [Section 2](#chapter2).


### 1.3 Broadcasting aka vectorization of transformers <a class="anchor" id="section_1_3"></a>

`sktime` transformers may be natively univariate, or apply only to a single time series.

Even if this is the case, they broadcast across variables and instances of time series, where applicable (als known as vectorization in `numpy` parlance).

This ensures that all `sktime` transformers can be applied to multivariate and multi-instance (panel, hierarchical) time series data.

Example 1: broadcasting/vectorization of time series to time series transformer

The `BoxCoxTransformer` from previous sections applies to single instances of univariate time series. When multiple instances or variables are seen, it broadcasts across both:

In [ ]:
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.utils._testing.hierarchical import _make_hierarchical

# hierarchical data with 2 variables and 2 levels
X = _make_hierarchical(n_columns=2)

X

In [ ]:
# constructing the transformers
boxcox_trafo = BoxCoxTransformer(method="mle")

# applying to X results in hierarchical data
boxcox_trafo.fit_transform(X)

Fitted model components of vectorized transformers can be found in the `transformers_` attribute, or accessed via the universal `get_fitted_params` interface:

In [ ]:
boxcox_trafo.transformers_
# this is a pandas.DataFrame that contains the fitted transformers
# one per time series instance and variable

In [ ]:
boxcox_trafo.get_fitted_params()
# this returns a dictionary
# the transformers DataFrame is available at the key "transformers"
# individual transformers are available at dataframe-like keys
# it also contains all fitted lambdas as keyed parameters

Example 2: broadcasting/vectorization of time series to scalar features transformer

The `SummaryTransformer` behaves similarly.
Multiple time series instances are transformed to different columns of the resulting data frame.

In [ ]:
from sktime.transformations.series.summarize import SummaryTransformer

summary_trafo = SummaryTransformer()

# this produces a pandas DataFrame with more rows and columns
# rows correspond to different instances in X
# columns are multiplied and names prefixed by [variablename]__
# there is one column per variable and transformed feature
summary_trafo.fit_transform(X)

### 1.4 Transformers as pipeline components <a class="anchor" id="section_1_4"></a>

A main use of `sktime` transformers is as pipeline components.

`sktime` transformers can be pipelined with any other `sktime` estimator type, including forecasters, classifiers, and other transformers.

Pipelines are estimators of the same type and retain all interface properties of the more specialized class.

The most universal pipeline operation is via `make_pipeline` or the `*` dunder.

Pipelining `pipe = trafo * est` or produces an estimator `pipe` of the same type as `est`.

In `pipe.fit`, first `trafo.fit_transform`, then `est.fit` is executed on the result.

In `pipe.othermethod`, first `trafo.transform`, then `est.othermethod` is executed.

(the arguments that are piped differ by type and can be looked up in the docstrings of pipeline classes, or specialized tutorials)

#### Example 1: pipeline of two transformers

In [ ]:
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.summarize import SummaryTransformer

pipe = Differencer() * SummaryTransformer()

# this constructs a TransformerPipeline, which is also a transformer
pipe

In [ ]:
from sktime.utils._testing.hierarchical import _make_hierarchical

X = _make_hierarchical(n_columns=2)

# this is a transformer with the same interface
# first applies differencer, then summary transform
pipe.fit_transform(X)

#### Example 2: forecaster pipeline

In [ ]:
from sktime.forecasting.naive import NaiveForecaster
from sktime.transformations.series.difference import Differencer

pipe = Differencer() * NaiveForecaster(strategy="drift")

# this constructs a TransformedTargetForecaster, which is also a forecaster
pipe

In [ ]:
from sktime.datasets import load_airline

y = load_airline()

# this is a forecaster with the same interface as NaiveForecaster
# first applies differencer, then naive forecaster, then inverts differencing
pipe.fit(y, fh=[1, 2, 3])
pipe.predict()

#### Example 3: classifier pipeline

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.transformations.series.exponent import ExponentTransformer

pipe = ExponentTransformer() * KNeighborsTimeSeriesClassifier()

# this constructs a ClassifierPipeline, which is also a classifier
pipe

In [ ]:
from sktime.datasets import load_unit_test

X_train, y_train = load_unit_test(split="TRAIN")
X_test, _ = load_unit_test(split="TEST")

# this is a forecaster with the same interface as knn-classifier
# first applies exponent transform, then knn-classifier
pipe.fit(X_train, y_train)
pipe.predict(X_test)

## 2. Transformers types and signatures <a class="anchor" id="chapter2"></a>

This section explains the different types of transformers found in `sktime` in detail.

There are four main types of transformation in `sktime`:

* transforming a series/sequence into scalar- or category-valued features. Examples: `tsfresh`, or extracting `mean` and `variance` overall.
* transforming a series into another series. Examples: detrending, smoothing, filtering, lagging.
* transforming a panel into another panel. Examples: principal component projection; applying individual series-to-series transformation to all series in the panel.
* transforming a pair of series into a scalar value. Examples: dynamic time warping distance between series/sequences; generalized alignment kernel between series/sequences.

Notably, the first three (series to primitive features, series to series, panel to panel) are covered by the same base class template and module. We call these transformers "time series transformers", or, simply, "transformers".
Kernels and distances for time series and sequences have the same mathematical signature and differ only in mathematical properties (e.g., definiteness assumptions) - they are covered by the more abstract scientific type of "pairwise transformer".

Below, we give an overview in sub-sections:
* reviewing common data container formats for series and panels
* showcasing the signature of time series transformers, that transform a single series or panel
* showcasing the signature of pairwise transformers, that transform a pair of series to a scalar, e.g., distances or kernels
* how to search `sktime` for transformers of a certain type

### 2.1 Data container format<a class="anchor" id="section_2_1"></a>

`sktime` transformers apply to individual time series and panels. Panels are collections of time series, and we refer to each time series in a panel as an "instance" of the panel.
This is formalized as abstract "scientific types" `Series` and `Panel`, with multiple possible in-memory representations, so-called "mtypes".

For the purpose of this tutorial, we will be working with the most common mtypes. For more details and formal data type specifications, see the "datatypes and datasets" tutorial.

`Series` are commonly represented as:

* `pandas.Series` for univariate time series and sequences
* `pandas.DataFrame` for uni- or multivariate time series and sequences

The `Series.index` and `DataFrame.index` are used for representing the time series or sequence index. `sktime` supports pandas integer, period and timestamp indices.

`Panel`-s are commonly represented as:
* a `pandas.DataFrame` in a specific format, defined by the `pd-multiindex` mtype. This has a 2-level index, for time points and instances
* a `list` of `pandas.DataFrame`, where all `pandas.DataFrame` are in the `Series` format. The different `list` elements are the different instances

In either case, the "time" index must be a `sktime` compatible time index type, as for `Series`.

In [ ]:
from sktime.datatypes import get_examples

In [ ]:
# example of a univariate series
get_examples("pd.Series", "Series")[0]

In [ ]:
# example of a multivariate series
get_examples("pd.DataFrame", "Series")[1]

In [ ]:
# example of a panel with mtype pd-multiindex
get_examples("pd-multiindex", "Panel")[0]

In [ ]:
# example of the same panel with mtype df-list
get_examples("df-list", "Panel")[0]

`sktime` supports more mtypes, see the "datatypes and datasets" tutorial for more details.

### 2.2 General transformer signature - time series transformers<a class="anchor" id="section_2_2"></a>

Transformers for `Series` and `Panel` have the same high-level interface. Depending which data type they are more commonly used for, they are found either in the `transformations.series` or `transformations.panel` module. As said, this does not imply a separate interface.

The most important interface points of transformers are:

1. construction with parameters, this is as with any other `sktime` estimator
2. fitting the transformer, via `fit`
3. transforming data, via `transform`
4. inverse transforming, via `inverse_transform` - not all transformers have this interface point, since not all are invertible
5. updating the transformer, via `update` - not all transformers have this interface point (`update` is currently work in progress, as of v0.8.x, contributions are appreciated)

We show this using two example transformers below - one whose `transform` outputs `Series`, and one whose `transform` outputs primitive features (numbers or categories).

We will apply both transformations to the following `Series` and `Panel` data:

In [ ]:
from sktime.datatypes import get_examples

# univariate series used in the examples
X_series = get_examples("pd.Series", "Series")[3]
# panel used in the examples
X_panel = get_examples("pd-multiindex", "Panel")[2]

In [ ]:
X_series

In [ ]:
X_panel

#### Example: transforming series to series

The Box-Cox transformer applies the Box-Cox transform to individual values in series or panels. At the start, the transformer needs to be constructed with parameter settings, this is the same as for any `sktime` estimator.

In [ ]:
# constructing the transformer
from sktime.transformations.series.boxcox import BoxCoxTransformer

my_boxcox_trafo = BoxCoxTransformer(method="mle")

Now, we apply the constructed transformer `my_trafo` to a (single, univariate) series. First, the transformer is fitted:

In [ ]:
# fitting the transformer
my_boxcox_trafo.fit(X_series)

Next, the transformer is applied, this results in a transformed series.

In [ ]:
# transforming the series
my_boxcox_trafo.transform(X_series)

Generally, the series passed to `transform` need not be the same as in `fit`, but if they are the same, the shorthand `fit_transform` can instead be used:

In [ ]:
my_boxcox_trafo.fit_transform(X_series)

The transformer can also be applied to `Panel` data.

In [ ]:
my_boxcox_trafo.fit_transform(X_panel)

Note: the `BoxCoxTransformer` used has applied the Box-Cox transform to every series in the panel individually,
but that need not be the case transformers in general.

#### Example: transforming series to primitive features

The summary transformer can be used to extract sample statistics such as mean and variance from a series. First, we construct the transformer:

In [ ]:
# constructing the transformer
from sktime.transformations.series.summarize import SummaryTransformer

my_summary_trafo = SummaryTransformer()

As before, we can fit/apply with `fit`, `transform`, and `fit_transform`.

`SummaryTransformer` returns primitive features, hence the output will be a `pandas.DataFrame`, each row corresponding to one series in the input. 

If the input is a single series, the output of `transform` and `fit_transform` will be a one-row, nine-column `DataFrame`, corresponding to the nine-number-summary of that one series:

In [ ]:
my_summary_trafo.fit_transform(X_series)

If the input is a panel, the output of `transform` and `fit_transform` will be a `DataFrame` with as many rows as the `Panel` had series. The `i`th row has the summary statistics of the `i`th series in the panel `X_panel`:

In [ ]:
my_summary_trafo.fit_transform(X_panel)

#### Transformers with series/panel output vs primitive output

Whether `transform` will return time series like objects (`Series` or `Panel`) or primitives (i.e. a `pandas.DataFrame`) can be checked by using the `"scitype:transform-output"` tag. This is `"Series"` for behaviour as in the first example (`BoxCoxTransformer`), and `"Primitives"` for behaviour as in the second example (`SummaryTransformer`):

In [ ]:
my_boxcox_trafo.get_tag("scitype:transform-output")

In [ ]:
my_summary_trafo.get_tag("scitype:transform-output")

Use of tags to characterize and search for transformers will be discussed in more detail in section 4.

NOTE: currently not all transformers are refactored to accept both `Series` to `Panel` arguments, the above may hence not fully work for all transformers. Contributions to the transformer refactor are very much appreciated.

### 2.3 General transformer signature - pairwise series transformers<a class="anchor" id="section_2_3"></a>

Pairwise series transformers model mathematical objects of signature `(Series, Series) -> float`, or, in mathematical notation, $$\texttt{series} \times\texttt{series}\rightarrow\mathbb{R}$$
Common examples are distances between series, or (positive definite) kernels on series.

Pairwise transformers have a parametric constructor, like any other `sktime` object. The transformation is achieved by the method `transform`, or, equivalently, for brevity, by a call to the constructed object.

The method `transform` always returns a 2D `numpy.ndarray`, and can be called in multiple ways:
* with two `Series` arguments `X, X2`, in which case a 1 x 1 array is returned. Denote this function by `t(X, X2)`
* with two `Panel` arguments `X`, `X2`, in which case an `m x n` array is returned, where `m` is the number of instances in `X` and `n` is the number of instances in `X2`. The `(i,j)`-th entry corresponds to `t(Xi, X2j)`, where `Xi` is ths `i`-th `Series` in the `Panel` `X`, and `X2j` is the `j`-th `Series` in the `Panel` `X2`.
* with one `Series` and one `Panel` argument, in which case the `Series` is interpreted as a 1-element `Panel`, with return as above.
* with one single argument, `Series` or `Panel`, in which case `X` and `X2` are assumed to be the same as the one argument, with behaviour as above.

We show these in a few examples below.

In [ ]:
from sktime.datatypes import get_examples

# unviariate series used in the examples
X_series = get_examples("pd.Series", "Series")[0]
X2_series = get_examples("pd.Series", "Series")[1]
# panel used in the examples
X_panel = get_examples("pd-multiindex", "Panel")[0]

First, we construct the pairwise transformer with parameters. In this case, the pairwise transformer is a distance (the mean Euclidean distance):

In [ ]:
# constructing the transformer
from sktime.dists_kernels import AggrDist, ScipyDist

# mean of paired Euclidean distances
my_series_dist = AggrDist(ScipyDist(metric="euclidean"))

We can then evaluate the distance by `transform` or direct call:

In [ ]:
# evaluate the metric on two series, via transform
my_series_dist.transform(X_series, X2_series)

In [ ]:
# evaluate the metric on two series, by direct call - this is the same
my_series_dist(X_series, X2_series)

In [ ]:
# evaluate the metric on two identical panels of three series
my_series_dist(X_panel, X_panel)

In [ ]:
# this is the same as providing only one argument
my_series_dist(X_panel)

In [ ]:
# one series, one panel
# we subset X_panel to univariate, since the distance in question
#     cannot compare series with different number of variables
my_series_dist(X_series, X_panel[["var_1"]])

Pairwise transformers are composable, and use the familiar `get_params` interface, just like any other `sktime` object and `scikit-learn` estimator:

In [ ]:
my_series_dist.get_params()

### 2.4 General transformer signature - pairwise transformers<a class="anchor" id="section_2_4"></a>

`sktime` also provides functionality for pairwise transformers on tabular data, i.e., mathematical objects of signature `(DataFrame-row, DataFrame-row) -> float`, or, in mathematical notation, $$\mathbb{R}^n \times\mathbb{R}^n\rightarrow\mathbb{R}$$.
Common examples are distances between series, or (positive definite) kernels on series.

The behaviour is as for series transformers, evaluation is callable by `transform(X, X2)` or a direct call.

Inputs to `transform` of a pairwise (tabular) transformer must always be `pandas.DataFrame`.
The output is an `m x n` matrix, a 2D `np.ndarray`, with `m = len(X), n=len(X2)`. 
The `(i,j)`-th entry corresponds to `t(Xi, X2j)`, where `Xi` is ths `i`-th row of `X`, and `X2j` is the `j`-th row of `X2`.
If `X2` is not passed, it defaults to `X`.

Example:

In [ ]:
from sktime.datatypes import get_examples

# we retrieve some DataFrame examples
X_tabular = get_examples("pd.DataFrame", "Series")[1]
X2_tabular = get_examples("pd.DataFrame", "Series")[1][0:3]

In [ ]:
# constructing the transformer
from sktime.dists_kernels import ScipyDist

# mean of paired Euclidean distances
my_tabular_dist = ScipyDist(metric="euclidean")

In [ ]:
# obtain matrix of distances between each pair of rows in X_tabular, X2_tabular
my_tabular_dist(X_tabular, X2_tabular)

### 2.5 Searching for transformers<a class="anchor" id="section_2_5"></a>

As with all `sktime` objects, we can use the `registry.all_estimators` utility to display all transformers in `sktime`.

The relevant scitypes are:
* `"transformer"` for all transformers (as in Section 2.2)
* `"transformer-pairwise"` for all pairwise transformers on tabular data (as in Section 2.4)
* `"transformer-panel"` for all pairwise transformers on panel data (as in Section 2.3)

To filter transformers (`"transformer"` scitype) further by input and output, use tags, most importantly:
* `"scitype:transform-output"` - the output scitype that the transform produces. `Series` for time series, `Primitives` for primitive features (float, categories).
* `"scitype:instancewise"` - whether transform uses all samples or acts by instance. If `True`, this is simply a vectorized operation per series. If `False`, then fitting on a single series does not have the same result as fitting on multiple.

These and further tags will be explained in more detail in Section 2.

In [ ]:
from sktime.registry import all_estimators

In [ ]:
# listing all pairwise panel transformers - distances, kernels on time series
all_estimators("transformer", as_dataframe=True)

In [ ]:
# now subset to transformers that extract scalar features
all_estimators(
    "transformer",
    as_dataframe=True,
    filter_tags={"scitype:transform-output": "Primitives"},
)

In [ ]:
# listing all pairwise (tabular) transformers - distances, kernels on vectors/df-rows
all_estimators("transformer-pairwise", as_dataframe=True)

In [ ]:
# listing all pairwise panel transformers - distances, kernels on time series
all_estimators("transformer-pairwise-panel", as_dataframe=True)

## 3. Transformers in pipelines and composition

work in progress. Contributions appreciated.

## 4. Extension guide - implementing your own transformer<a class="anchor" id="chapter4"></a>

`sktime` is meant to be easily extensible, for direct contribution to `sktime` as well as for local/private extension with custom methods.

To get started:

* Follow the ["implementing estimator" developer guide](https://www.sktime.org/en/stable/developer_guide/add_estimators.html)
* Use the [simple forecasting extension template](https://github.com/sktime/sktime/blob/main/extension_templates/transformer_simple.py) for transformers without stream, probabilistic, or hierarchical functionality
* Use the [advanced forecasting extension template](https://github.com/sktime/sktime/blob/main/extension_templates/transformer.py) for transformers with stream, probabilistic or hierarchical functionality
* For pairwise transformers, use the [time series pairwise transformer extension template](https://github.com/sktime/sktime/blob/main/extension_templates/dist_kern_panel.py) or the [scalar/tabular pairwise transformer extension template](https://github.com/sktime/sktime/blob/main/extension_templates/dist_kern_tab.py)

1. read through the [transformer extension template](https://github.com/alan-turing-institute/sktime/blob/main/extension_templates/transformer.py) - this is a `python` file with `todo` blocks that mark the places in which changes need to be added.
2. optionally, if you are planning any major surgeries to the interface: look at the [base class architecture](https://github.com/alan-turing-institute/sktime/blob/main/sktime/transformations/base.py) - note that "ordinary" extension (e.g., new algorithm) should be easily doable without this.
3. copy the correct transformer extension template to a local folder in your own repository (local/private extension), or to a suitable location in your clone of the `sktime` or affiliated repository (if contributed extension), inside `sktime.transformations`; rename the file and update the file docstring appropriately.
4. address the "todo" parts. Usually, this means: changing the name of the class, setting the tag values, specifying hyper-parameters, filling in `__init__`, `_fit`, `_transform`, and optional methods such as `_inverse_transform` or `_update` (for details see the extension template). You can add private methods as long as they do not override the default public interface. For more details, see the extension template.
5. to test your estimator manually: import your estimator and run it in the worfklows in Section 1; then use it in the compositors in Section 3.
6. To test your estimator automatically: call `sktime.utils.estimator_checks.check_estimator` on your estimator. You can call this on a class or object instance. Ensure you have specified test parameters in the `get_test_params` method, according to the extension template.

In case of direct contribution to `sktime` or one of its affiliated packages, additionally:
* add yourself as an author to the code, and to the `CODEOWNERS` for the new estimator file(s).
* create a pull request that contains only the new estimators (and their inheritance tree, if it's not just one class), as well as the automated tests as described above.
* in the pull request, describe the estimator and optimally provide a publication or other technical reference for the strategy it implements.
* before making the pull request, ensure that you have all necessary permissions to contribute the code to a permissive license (BSD-3) open source project.

## 5. Summary<a class="anchor" id="chapter5"></a>

* `sktime` comes with transformation algorithms (or transformers), all of which share a common interface. The interface is fully interoperable with the `scikit-learn` interface.

* Transformers exist in several categories: series being transformed to series; series being transformed to primitive features (floats, categories); pairwise transformers where pairs of series or vectors are transformed to a float output, such as distance functions and kernel functions.

* Transformers are typically used as components of other algorithms across learning tasks, for instance as feature extraction steps in pipelines, or as distances in a distance-based classification algorithm. Composition using `sktime` transformers is fully modular.

* `sktime` provides easy-to-use extension templates for all the above.